In [1]:
import pandas

In [2]:
def load_field_data(save):
    rating = pandas.read_csv("./rawdata/{}/players_fielding.csv".format(save))
    stat = pandas.read_csv("./rawdata/{}/players_career_fielding_stats.csv".format(save))
    return stat.merge(rating,on='player_id',suffixes=('_st','_r'))

In [3]:
data = load_field_data('test_8001.lg')

In [4]:
data

,player_id,year,team_id_st,league_id_st,level_id,split_id,position_st,tc,a,po,...,fielding_experience9,fielding_rating_pos1,fielding_rating_pos2,fielding_rating_pos3,fielding_rating_pos4,fielding_rating_pos5,fielding_rating_pos6,fielding_rating_pos7,fielding_rating_pos8,fielding_rating_pos9
0,1,2023,8,200,1,0,1,27,18,8,...,0,132,0,0,0,0,0,0,0,0
1,2,2023,20,201,2,0,1,10,7,3,...,0,69,0,0,0,0,0,0,0,0
2,3,2023,14,200,1,0,1,16,13,3,...,0,119,0,0,0,0,0,0,0,0
3,3,2023,29,201,2,0,1,2,1,1,...,0,119,0,0,0,0,0,0,0,0
4,4,2023,8,200,1,0,1,3,2,1,...,0,135,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7400,3311,2023,87,205,6,0,1,0,0,0,...,0,142,0,0,0,0,0,0,0,0
7401,3312,2023,75,204,5,0,1,0,0,0,...,0,82,0,0,0,0,0,0,0,0
7402,3312,2023,92,205,6,0,1,13,9,4,...,0,82,0,0,0,0,0,0,0,0
7403,3313,2023,57,203,4,0,1,0,0,0,...,0,115,0,0,0,0,0,0,0,0


In [5]:
print(data.columns)

Index(['player_id', 'year', 'team_id_st', 'league_id_st', 'level_id',
       'split_id', 'position_st', 'tc', 'a', 'po', 'er', 'ip', 'g', 'gs', 'e',
       'dp', 'tp', 'pb', 'sba', 'rto', 'ipf', 'plays', 'plays_base', 'roe',
       'opps_0', 'opps_made_0', 'opps_1', 'opps_made_1', 'opps_2',
       'opps_made_2', 'opps_3', 'opps_made_3', 'opps_4', 'opps_made_4',
       'opps_5', 'opps_made_5', 'framing', 'arm', 'zr', 'team_id_r',
       'league_id_r', 'position_r', 'role', 'fielding_ratings_infield_range',
       'fielding_ratings_infield_arm', 'fielding_ratings_turn_doubleplay',
       'fielding_ratings_outfield_range', 'fielding_ratings_outfield_arm',
       'fielding_ratings_catcher_arm', 'fielding_ratings_catcher_ability',
       'fielding_ratings_infield_error', 'fielding_ratings_outfield_error',
       'fielding_experience0', 'fielding_experience1', 'fielding_experience2',
       'fielding_experience3', 'fielding_experience4', 'fielding_experience5',
       'fielding_experience6',

In [6]:
def conv(x): # 1-200 scale to 20-80
    if type(x) is str:
        x = int(x)
    r = x*60.0/200+20 # 0 -> 20, 200 -> 80
    return int(round(r/5)*5)

In [7]:
conv(199)

80

In [21]:
import pandas as pd
total_data = pd.concat([load_field_data('test_{}.lg'.format(sv)) for sv in range(8001, 8016)])
total_data['range_vis'] = total_data.apply(lambda row: conv(row['fielding_ratings_outfield_range']) if row['position_st'] >=7 else conv(row['fielding_ratings_infield_range']), axis=1)
agg = total_data.groupby(['position_st', 'range_vis'], as_index=False)[['opps_0', 'opps_made_0', 'opps_1', 'opps_made_1', 'opps_2',
       'opps_made_2', 'opps_3', 'opps_made_3', 'opps_4', 'opps_made_4',
       'opps_5', 'opps_made_5']].sum()
for t in range(6):
    agg['opps_{}_pct'.format(t)] = agg.apply(lambda row: row['opps_made_{}'.format(t)]/row['opps_{}'.format(t)], axis=1)
agg

C:\Users\goreh\AppData\Local\Temp\ipykernel_39744\316640046.py:8: RuntimeWarning: invalid value encountered in scalar divide
  agg['opps_{}_pct'.format(t)] = agg.apply(lambda row: row['opps_made_{}'.format(t)]/row['opps_{}'.format(t)], axis=1)
C:\Users\goreh\AppData\Local\Temp\ipykernel_39744\316640046.py:8: RuntimeWarning: invalid value encountered in scalar divide
  agg['opps_{}_pct'.format(t)] = agg.apply(lambda row: row['opps_made_{}'.format(t)]/row['opps_{}'.format(t)], axis=1)
C:\Users\goreh\AppData\Local\Temp\ipykernel_39744\316640046.py:8: RuntimeWarning: invalid value encountered in scalar divide
  agg['opps_{}_pct'.format(t)] = agg.apply(lambda row: row['opps_made_{}'.format(t)]/row['opps_{}'.format(t)], axis=1)
C:\Users\goreh\AppData\Local\Temp\ipykernel_39744\316640046.py:8: RuntimeWarning: invalid value encountered in scalar divide
  agg['opps_{}_pct'.format(t)] = agg.apply(lambda row: row['opps_made_{}'.format(t)]/row['opps_{}'.format(t)], axis=1)
C:\Users\goreh\AppData\L

,position_st,range_vis,opps_0,opps_made_0,opps_1,opps_made_1,opps_2,opps_made_2,opps_3,opps_made_3,opps_4,opps_made_4,opps_5,opps_made_5,opps_0_pct,opps_1_pct,opps_2_pct,opps_3_pct,opps_4_pct,opps_5_pct
0,1,20,2,2,0,0,3,3,0,0,0,0,0,0,1.000000,NaN,1.000000,NaN,NaN,NaN
1,1,25,2281,1856,600,340,388,246,10,0,98,0,0,0,0.813678,0.566667,0.634021,0.000000,0.000000,NaN
2,1,30,29898,24703,7503,4193,5149,3269,89,0,845,0,0,0,0.826243,0.558843,0.634881,0.000000,0.000000,NaN
3,1,35,25972,21916,6486,3807,4442,3090,72,0,495,0,0,0,0.843832,0.586957,0.695633,0.000000,0.000000,NaN
4,1,40,8213,6946,2135,1283,1366,961,21,0,117,0,0,0,0.845732,0.600937,0.703514,0.000000,0.000000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104,9,55,66881,63807,13779,9059,11327,4689,8755,2367,8843,1676,16566,21,0.954038,0.657450,0.413967,0.270360,0.189528,0.001268
105,9,60,53236,51219,10875,7453,8727,3818,6732,1903,6929,1401,13151,20,0.962112,0.685333,0.437493,0.282680,0.202194,0.001521
106,9,65,46468,44921,9350,6475,7566,3432,5726,1687,5990,1244,11314,26,0.966708,0.692513,0.453608,0.294621,0.207679,0.002298
107,9,70,19285,18705,3883,2780,3241,1553,2345,700,2411,550,4441,11,0.969925,0.715941,0.479173,0.298507,0.228121,0.002477


In [30]:
pos = ['', 'P','C','1B','2B','3B','SS','LF','CF','RF']

In [31]:
from IPython.display import display, HTML
# for corner IF
for p in [3,5]:
    df = agg[agg['position_st']==p].set_index('range_vis')
    df = df[['opps_0', 'opps_0_pct','opps_1', 'opps_1_pct','opps_2', 'opps_2_pct','opps_3', 'opps_3_pct','opps_4', 'opps_4_pct','opps_5', 'opps_5_pct' ]]
    print ('for ', pos[p])
    display(df)

for  1B


,opps_0,opps_0_pct,opps_1,opps_1_pct,opps_2,opps_2_pct,opps_3,opps_3_pct,opps_4,opps_4_pct,opps_5,opps_5_pct
range_vis,,,,,,,,,,,,
20,126,0.928571,39,0.820513,27,0.518519,17,0.176471,6,0.000000,9,0.0
25,5132,0.918355,1038,0.812139,818,0.376528,477,0.253669,476,0.088235,503,0.0
30,21641,0.927591,4450,0.835506,3619,0.435756,2060,0.245631,1932,0.115942,2334,0.0
35,38574,0.934178,8057,0.853419,6286,0.449889,3627,0.284808,3473,0.138497,4150,0.0
40,33328,0.940951,7042,0.869071,5401,0.475097,3214,0.305849,2960,0.158108,3602,0.0
45,32364,0.945155,6504,0.882841,5274,0.509101,2967,0.344119,2788,0.185796,3691,0.0
50,27392,0.948562,5526,0.885993,4405,0.496027,2570,0.340078,2392,0.182274,2923,0.0
55,15501,0.947423,3189,0.886485,2488,0.505225,1561,0.325432,1328,0.189006,1767,0.0
60,5028,0.951273,1042,0.904990,813,0.520295,451,0.325942,425,0.202353,595,0.0


for  3B


,opps_0,opps_0_pct,opps_1,opps_1_pct,opps_2,opps_2_pct,opps_3,opps_3_pct,opps_4,opps_4_pct,opps_5,opps_5_pct
range_vis,,,,,,,,,,,,
20,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN
25,14,0.714286,7,0.857143,2,0.000000,0,NaN,2,0.000000,2,0.0
30,57,0.842105,27,0.703704,20,0.550000,7,0.714286,9,0.111111,6,0.0
35,997,0.877633,397,0.773300,241,0.601660,152,0.467105,171,0.134503,121,0.0
40,5530,0.926401,2084,0.805662,1342,0.628912,816,0.528186,954,0.227463,777,0.0
45,19309,0.926615,7525,0.826977,4817,0.627984,2796,0.528612,3421,0.272143,2604,0.0
50,44007,0.934738,16834,0.834145,10959,0.653344,6496,0.543719,7667,0.281857,6060,0.0
55,47478,0.940330,18319,0.846771,11804,0.673331,6916,0.558704,8138,0.305849,6370,0.0
60,36952,0.939273,14064,0.851962,9123,0.670394,5417,0.564150,6631,0.313075,5168,0.0


In [32]:
# for core IF
for p in [4,6]:
    df = agg[agg['position_st']==p].set_index('range_vis')
    df = df[['opps_0', 'opps_0_pct','opps_1', 'opps_1_pct','opps_2', 'opps_2_pct','opps_3', 'opps_3_pct','opps_4', 'opps_4_pct','opps_5', 'opps_5_pct' ]]
    print ('for ', pos[p])
    display(df)

for  2B


,opps_0,opps_0_pct,opps_1,opps_1_pct,opps_2,opps_2_pct,opps_3,opps_3_pct,opps_4,opps_4_pct,opps_5,opps_5_pct
range_vis,,,,,,,,,,,,
25,25,0.480000,1,0.000000,4,0.000000,7,0.000000,1,0.000000,9,0.0
30,56,0.785714,12,0.666667,10,0.100000,14,0.071429,11,0.272727,25,0.0
35,244,0.750000,83,0.506024,81,0.234568,65,0.046154,65,0.123077,132,0.0
40,1746,0.852234,386,0.481865,388,0.247423,485,0.148454,493,0.154158,771,0.0
45,11159,0.871404,2747,0.512559,2817,0.323749,3166,0.171826,3106,0.169994,5123,0.0
50,36627,0.886122,9323,0.567414,9047,0.359125,10424,0.189083,10540,0.189848,17023,0.0
55,63295,0.901556,16029,0.608647,15540,0.399228,18230,0.221942,18003,0.221852,29125,0.0
60,73697,0.913755,19021,0.638978,18273,0.448366,21149,0.235188,21186,0.237704,33961,0.0
65,59754,0.929176,15195,0.683317,14596,0.478145,17102,0.253596,16807,0.261558,26945,0.0


for  SS


,opps_0,opps_0_pct,opps_1,opps_1_pct,opps_2,opps_2_pct,opps_3,opps_3_pct,opps_4,opps_4_pct,opps_5,opps_5_pct
range_vis,,,,,,,,,,,,
20,2,0.500000,1,0.000000,0,NaN,0,NaN,1,0.000000,0,NaN
25,84,0.738095,12,0.416667,24,0.250000,25,0.320000,28,0.035714,27,0.0
30,299,0.692308,56,0.357143,63,0.206349,62,0.290323,71,0.140845,120,0.0
35,448,0.691964,66,0.348485,99,0.222222,126,0.158730,107,0.130841,166,0.0
40,457,0.761488,94,0.372340,111,0.252252,117,0.188034,119,0.058824,175,0.0
45,3081,0.801039,509,0.410609,750,0.270667,709,0.162200,756,0.129630,1035,0.0
50,13888,0.850086,2488,0.417203,3395,0.307216,3411,0.215186,3586,0.141383,4974,0.0
55,28101,0.874275,4962,0.444377,6850,0.336204,6617,0.249962,7406,0.190656,10006,0.0
60,59077,0.895171,10632,0.494357,14601,0.388740,13997,0.269558,15640,0.226790,20564,0.0


In [34]:
# for corner OF 
for p in [7,9]:
    df = agg[agg['position_st']==p].set_index('range_vis')
    df = df[['opps_0', 'opps_0_pct','opps_1', 'opps_1_pct','opps_2', 'opps_2_pct','opps_3', 'opps_3_pct','opps_4', 'opps_4_pct','opps_5', 'opps_5_pct' ]]
    print ('for ', pos[p])
    display(df)

for  LF


,opps_0,opps_0_pct,opps_1,opps_1_pct,opps_2,opps_2_pct,opps_3,opps_3_pct,opps_4,opps_4_pct,opps_5,opps_5_pct
range_vis,,,,,,,,,,,,
20,3,1.000000,0,NaN,0,NaN,0,NaN,0,NaN,0,NaN
25,36,0.972222,9,0.777778,9,0.444444,4,0.250000,4,0.250000,7,0.000000
30,230,0.908696,46,0.500000,33,0.333333,29,0.206897,32,0.156250,60,0.000000
35,1818,0.934543,375,0.613333,328,0.283537,248,0.254032,270,0.129630,471,0.000000
40,25594,0.937837,5260,0.591445,4286,0.333178,3303,0.231910,3528,0.138889,6047,0.000165
45,60763,0.945592,12571,0.617135,10192,0.355867,7752,0.249484,7919,0.167950,13946,0.000359
50,58613,0.949789,12027,0.634822,9804,0.382905,7455,0.254863,7712,0.173626,13184,0.001745
55,57145,0.956584,11718,0.667520,9558,0.425821,7175,0.280418,7627,0.187885,12864,0.001011
60,75105,0.961667,15452,0.690072,12243,0.452340,9480,0.284072,9957,0.208597,16634,0.002164


for  RF


,opps_0,opps_0_pct,opps_1,opps_1_pct,opps_2,opps_2_pct,opps_3,opps_3_pct,opps_4,opps_4_pct,opps_5,opps_5_pct
range_vis,,,,,,,,,,,,
20,3,1.000000,0,NaN,1,0.000000,0,NaN,1,0.000000,1,0.000000
25,57,0.982456,10,0.400000,9,0.222222,9,0.333333,11,0.181818,15,0.000000
30,40,0.950000,7,0.142857,3,0.333333,1,1.000000,3,0.333333,10,0.000000
35,1460,0.920548,285,0.536842,262,0.270992,191,0.219895,172,0.122093,410,0.000000
40,24214,0.930990,5076,0.575256,4103,0.302218,3167,0.223240,3231,0.128443,6057,0.000000
45,60594,0.940357,12649,0.602261,10158,0.335696,7740,0.240052,8157,0.150791,15096,0.000397
50,72582,0.946901,14836,0.629954,12026,0.374771,9016,0.239685,9415,0.166861,17654,0.000227
55,66881,0.954038,13779,0.657450,11327,0.413967,8755,0.270360,8843,0.189528,16566,0.001268
60,53236,0.962112,10875,0.685333,8727,0.437493,6732,0.282680,6929,0.202194,13151,0.001521


In [35]:
# for CF
for p in [8]:
    df = agg[agg['position_st']==p].set_index('range_vis')
    df = df[['opps_0', 'opps_0_pct','opps_1', 'opps_1_pct','opps_2', 'opps_2_pct','opps_3', 'opps_3_pct','opps_4', 'opps_4_pct','opps_5', 'opps_5_pct' ]]
    print ('for ', pos[p])
    display(df)

for  CF


,opps_0,opps_0_pct,opps_1,opps_1_pct,opps_2,opps_2_pct,opps_3,opps_3_pct,opps_4,opps_4_pct,opps_5,opps_5_pct
range_vis,,,,,,,,,,,,
20,18,0.833333,6,0.833333,2,0.000000,1,0.000000,2,0.000000,1,0.000000
25,60,0.966667,9,0.888889,7,0.285714,5,0.000000,4,0.250000,16,0.000000
30,111,0.972973,26,0.846154,20,0.400000,8,0.125000,20,0.050000,23,0.000000
35,137,0.963504,28,0.750000,22,0.272727,8,0.000000,24,0.083333,27,0.000000
40,189,0.952381,28,0.785714,27,0.444444,21,0.238095,26,0.000000,37,0.000000
45,1688,0.963863,324,0.709877,187,0.315508,123,0.203252,239,0.079498,314,0.000000
50,13031,0.966465,2544,0.730739,1610,0.340373,1088,0.169118,2043,0.098385,2463,0.000000
55,31198,0.972530,6052,0.730998,3910,0.399488,2606,0.196853,5047,0.105013,5907,0.000169
60,90924,0.979026,17394,0.775555,11294,0.450151,7516,0.224987,14169,0.129649,16923,0.000295


In [38]:
total_by_pos = total_data.groupby('position_st')[['opps_0', 'opps_made_0', 'opps_1', 'opps_made_1', 'opps_2',
       'opps_made_2', 'opps_3', 'opps_made_3', 'opps_4', 'opps_made_4',
       'opps_5', 'opps_made_5']].sum()
for t in range(6):
    total_by_pos['opps_{}_pct'.format(t)] = total_by_pos.apply(lambda row: row['opps_made_{}'.format(t)]/row['opps_{}'.format(t)], axis=1)
total_by_pos[['opps_0_pct','opps_1_pct','opps_2_pct','opps_3_pct','opps_4_pct','opps_5_pct']]

C:\Users\goreh\AppData\Local\Temp\ipykernel_39744\3045191902.py:5: RuntimeWarning: invalid value encountered in scalar divide
  total_by_pos['opps_{}_pct'.format(t)] = total_by_pos.apply(lambda row: row['opps_made_{}'.format(t)]/row['opps_{}'.format(t)], axis=1)


,opps_0_pct,opps_1_pct,opps_2_pct,opps_3_pct,opps_4_pct,opps_5_pct
position_st,,,,,,
1,0.836082,0.577422,0.668941,0.000000,0.000000,NaN
2,0.842035,0.900856,0.708337,0.000000,0.000000,0.000000
3,0.940169,0.867749,0.476047,0.307393,0.159152,0.000000
4,0.911469,0.635429,0.433485,0.231988,0.235812,0.000000
5,0.936958,0.842893,0.663411,0.554000,0.299215,0.000000
6,0.909486,0.542384,0.452995,0.297789,0.263666,0.000000
7,0.954532,0.657519,0.408671,0.266843,0.186037,0.001531
8,0.984617,0.806425,0.509661,0.249492,0.156279,0.000676
9,0.952274,0.647847,0.396014,0.261879,0.179805,0.001047


In [42]:
total_opps = total_by_pos[['opps_0','opps_1','opps_2','opps_3','opps_4']].sum().sum()